In [2]:
!git clone https://github.com/MikolajZielinski/genie.git
%cd genie

Cloning into 'genie'...
remote: Enumerating objects: 113, done.
remote: Counting objects: 100% (61/61), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 113 (delta 25), reused 23 (delta 5), pack-reused 52 (from 1)
Receiving objects: 100% (113/113), 52.82 MiB | 34.30 MiB/s, done.
Resolving deltas: 100% (31/31), done.
/kaggle/working/genie


In [3]:
!pip install nerfstudio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 2.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 580.8/580.8 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 109.6 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 40.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.0/510.0 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 79.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4

In [15]:
# Create the target directory first
!mkdir -p ./optix_install

# Try running the installer with the directory already created
!./NVIDIA-OptiX-SDK-8.1.0-linux64-x86_64-35015278.sh --skip-license --prefix=./optix_install

# If that still fails, try extracting manually (the .sh is actually a tar.gz inside)
!tail -n +$(grep -n -a "^__END_HEADER__" NVIDIA-OptiX-SDK-8.1.0-linux64-x86_64-35015278.sh | cut -d: -f1) NVIDIA-OptiX-SDK-8.1.0-linux64-x86_64-35015278.sh | tar xzv

# Check what was extracted
!ls -la

NVIDIA OptiX Installer Version: 8.1.0, Copyright (c) NVIDIA Corporation
This is a self-extracting archive.
The archive will be extracted to: ./optix_install

Using target directory: ./optix_install
Extracting, please wait...

Unpacking finished successfully
tail: invalid number of lines: ‘+’

gzip: stdin: unexpected end of file
tar: Child returned status 1
tar: Error is not recoverable: exiting now
total 55844
drwxr-xr-x 8 root root     4096 Nov 29 10:24 .
drwxr-xr-x 4 root root     4096 Nov 29 10:08 ..
drwxr-xr-x 3 root root     4096 Nov 29 10:08 blender
drwxr-xr-x 2 root root     4096 Nov 29 10:08 docker
drwxr-xr-x 7 root root     4096 Nov 29 10:08 genie
drwxr-xr-x 8 root root     4096 Nov 29 10:08 .git
-rw-r--r-- 1 root root      185 Nov 29 10:08 .gitignore
drwxr-xr-x 2 root root     4096 Nov 29 10:08 images
-rw-r--r-- 1 root root     1076 Nov 29 10:08 LICENSE
-rwxr-xr-x 1 root root 57124192 Nov 29 10:23 NVIDIA-OptiX-SDK-8.1.0-linux64-x86_64-35015278.sh
drwxr-xr-x 5 root root     40

In [16]:
# Check what was extracted
!ls -la optix_install/

# Look for the SDK directory structure
!find optix_install/ -name "include" -type d

# Check for optix.h header
!find optix_install/ -name "optix.h"

total 384
drwxr-xr-x  5 root root   4096 Nov 29 10:24 .
drwxr-xr-x  8 root root   4096 Nov 29 10:24 ..
drwxr-xr-x  2 root root   4096 Oct 21  2024 doc
drwxr-xr-x  3 root root   4096 Oct 21  2024 include
-rw-r--r--  1 root root 355775 Oct 21  2024 OptiX_EndUserLicense.pdf
-rw-r--r--  1 root root  14011 Oct 21  2024 OptiX_ThirdParty_Licenses.txt
drwxr-xr-x 37 root root   4096 Oct 21  2024 SDK
optix_install/SDK/support/GLFW/include
optix_install/SDK/optixMixSDKs/OptiX SDK 7.5.0/include
optix_install/include
optix_install/SDK/optixMixSDKs/OptiX SDK 7.5.0/include/optix.h
optix_install/include/optix.h


In [17]:
# The build script expects: NVIDIA-OptiX-SDK-8.1.0-linux64-x86_64/include
# So let's create that structure

# Move the extracted SDK to the expected name
!mv optix_install NVIDIA-OptiX-SDK-8.1.0-linux64-x86_64

# Move it to genie/knn/
!mv NVIDIA-OptiX-SDK-8.1.0-linux64-x86_64 genie/knn/

# Verify it's in the right place
!ls genie/knn/

# Verify the include directory
!ls genie/knn/NVIDIA-OptiX-SDK-8.1.0-linux64-x86_64/include/

# Check for optix.h specifically
!ls genie/knn/NVIDIA-OptiX-SDK-8.1.0-linux64-x86_64/include/optix.h

bindings.cpp	knn_algorithms.py		       shaders.cu
build_optix.sh	KNN.cu
Header.cuh	NVIDIA-OptiX-SDK-8.1.0-linux64-x86_64
internal			   optix_function_table.h  optix_stack_size.h
optix_denoiser_tiling.h		   optix.h		   optix_stubs.h
optix_device.h			   optix_host.h		   optix_types.h
optix_function_table_definition.h  optix_micromap.h
genie/knn/NVIDIA-OptiX-SDK-8.1.0-linux64-x86_64/include/optix.h


In [26]:
%%bash
cat > build_optix_fixed.sh << 'EOF'
#!/bin/bash

set -e

echo "🔍 Detecting GPU Compute Capability..."
CC=$(nvidia-smi --query-gpu=compute_cap --format=csv,noheader | head -n1)
CUDA_ARCH="sm_${CC/./}"
echo "Detected CUDA_ARCH=$CUDA_ARCH"

echo "🔍 Detecting CUDA include path..."
CUDA_INCLUDE=$(dirname $(dirname $(which nvcc)))/include
echo "Detected CUDA include path: $CUDA_INCLUDE"

echo "🔍 Detecting Thrust include path..."
CUDA_ROOT=$(dirname $(dirname $(which nvcc)))
THRUST_INCLUDE="$CUDA_ROOT/include"
if [ ! -d "$THRUST_INCLUDE/thrust" ]; then
    THRUST_INCLUDE="$CUDA_ROOT/targets/x86_64-linux/include"
fi
echo "Detected Thrust include path: $THRUST_INCLUDE"

LIBCUDA_PATH=$(find /usr -name 'libcuda.so*' 2>/dev/null | head -n 1)
if [ -n "$LIBCUDA_PATH" ]; then
  CUDA_LIB_DIR=$(dirname "$LIBCUDA_PATH")
  echo "Found libcuda.so at $CUDA_LIB_DIR"
else
  echo "libcuda.so not found!"
  CUDA_LIB_DIR=""
fi

OPTIX_INCLUDE="NVIDIA-OptiX-SDK-8.1.0-linux64-x86_64/include"
PYTHON_BIN=python3
PYTORCH_DIR=$($PYTHON_BIN -c "import torch, os; print(os.path.join(torch.__path__[0], 'include'))")
PYTORCH_API_DIR="$PYTORCH_DIR/torch/csrc/api/include"
PYTHON_SITE_PACKAGES=$($PYTHON_BIN -c "import site; print(site.getsitepackages()[0])")
TORCH_LIB_DIR="$PYTHON_SITE_PACKAGES/torch/lib"
PYBIND11_INCLUDES=$($PYTHON_BIN -m pybind11 --includes)

echo "🔍 Detecting PyTorch ABI flag..."
TORCH_CXX11_ABI=$($PYTHON_BIN -c "import torch; print(int(torch._C._GLIBCXX_USE_CXX11_ABI))")
if [ "$TORCH_CXX11_ABI" = "1" ]; then
    ABI_FLAG="-D_GLIBCXX_USE_CXX11_ABI=1"
    echo "Detected ABI: CXX11 ABI enabled"
else
    ABI_FLAG="-D_GLIBCXX_USE_CXX11_ABI=0"
    echo "Detected ABI: CXX11 ABI disabled (pre-CXX11)"
fi
CXX_STD="-std=c++17"

BUILD_DIR="build"
mkdir -p $BUILD_DIR

echo "📦 Compiling shaders.cu to PTX..."
nvcc -ptx -arch=${CUDA_ARCH} -o ${BUILD_DIR}/shaders.ptx shaders.cu \
  -I${OPTIX_INCLUDE} -I${CUDA_INCLUDE} ${CXX_STD}

echo "🔧 Compiling KNN.cu..."
nvcc -Xcompiler -fPIC -c KNN.cu -o ${BUILD_DIR}/KNN.o \
  --gpu-architecture=compute_60 --gpu-code=${CUDA_ARCH} \
  -I${OPTIX_INCLUDE} -I${CUDA_INCLUDE} ${ABI_FLAG} ${CXX_STD}

echo "🔗 Compiling bindings with nvcc..."
nvcc -shared -Xcompiler -fPIC bindings.cpp ${BUILD_DIR}/KNN.o -o optix_knn.so \
  ${CXX_STD} ${ABI_FLAG} ${PYBIND11_INCLUDES} \
  -I${CUDA_INCLUDE} \
  -I${OPTIX_INCLUDE} \
  -I${THRUST_INCLUDE} \
  -I${PYTORCH_DIR} \
  -I${PYTORCH_API_DIR} \
  -L${TORCH_LIB_DIR} \
  -L${CUDA_LIB_DIR} \
  -ltorch -ltorch_cpu -ltorch_python -lc10 -lcuda \
  -Xlinker -rpath=${TORCH_LIB_DIR}

echo "✅ Build complete. Output: optix_knn.so"
EOF

chmod +x build_optix_fixed.sh
./build_optix_fixed.sh

🔍 Detecting GPU Compute Capability...
Detected CUDA_ARCH=sm_60
🔍 Detecting CUDA include path...
Detected CUDA include path: /usr/local/cuda/include
🔍 Detecting Thrust include path...
Detected Thrust include path: /usr/local/cuda/include
Found libcuda.so at /usr/local/nvidia/lib64
🔍 Detecting PyTorch ABI flag...
Detected ABI: CXX11 ABI disabled (pre-CXX11)
📦 Compiling shaders.cu to PTX...
🔧 Compiling KNN.cu...
🔗 Compiling bindings with nvcc...


shaders.cu(9): warning #20044-D: extern declaration of the entity optixLaunchParams is treated as a static definition

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

KNN.cu(467): warning #68-D: integer conversion resulted in a change of sign
    float *base_address_2 = &tmp[((threadIdx.x & -32) * 20) + ((threadIdx.x & -32) >> 3)];
                                                 ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

KNN.cu(467): warning #68-D: integer conversion resulted in a change of sign
    float *base_address_2 = &tmp[((threadIdx.x & -32) * 20) + ((threadIdx.x & -32) >> 3)];
                                                                              ^

KNN.cu(467): warning #68-D: integer conversion resulted in a change of sign
    float *base_address_2 = &tmp[((threadIdx.x & -32) * 20) + ((threadIdx.x & -32) >> 3)];
                                                 ^

Remark: The warnings can be suppresse

CalledProcessError: Command 'b'cat > build_optix_fixed.sh << \'EOF\'\n#!/bin/bash\n\nset -e\n\necho "\xf0\x9f\x94\x8d Detecting GPU Compute Capability..."\nCC=$(nvidia-smi --query-gpu=compute_cap --format=csv,noheader | head -n1)\nCUDA_ARCH="sm_${CC/./}"\necho "Detected CUDA_ARCH=$CUDA_ARCH"\n\necho "\xf0\x9f\x94\x8d Detecting CUDA include path..."\nCUDA_INCLUDE=$(dirname $(dirname $(which nvcc)))/include\necho "Detected CUDA include path: $CUDA_INCLUDE"\n\necho "\xf0\x9f\x94\x8d Detecting Thrust include path..."\nCUDA_ROOT=$(dirname $(dirname $(which nvcc)))\nTHRUST_INCLUDE="$CUDA_ROOT/include"\nif [ ! -d "$THRUST_INCLUDE/thrust" ]; then\n    THRUST_INCLUDE="$CUDA_ROOT/targets/x86_64-linux/include"\nfi\necho "Detected Thrust include path: $THRUST_INCLUDE"\n\nLIBCUDA_PATH=$(find /usr -name \'libcuda.so*\' 2>/dev/null | head -n 1)\nif [ -n "$LIBCUDA_PATH" ]; then\n  CUDA_LIB_DIR=$(dirname "$LIBCUDA_PATH")\n  echo "Found libcuda.so at $CUDA_LIB_DIR"\nelse\n  echo "libcuda.so not found!"\

In [27]:
# Rename bindings.cpp to bindings.cu temporarily and compile
!cp bindings.cpp bindings_backup.cpp
!mv bindings.cpp bindings.cu

# Now try compiling with the renamed file
!nvcc -shared -Xcompiler -fPIC bindings.cu build/KNN.o -o optix_knn.so \
  -std=c++17 -D_GLIBCXX_USE_CXX11_ABI=0 \
  $(python3 -m pybind11 --includes) \
  -I/usr/local/cuda/include \
  -INVIDIA-OptiX-SDK-8.1.0-linux64-x86_64/include \
  -I/usr/local/cuda/include \
  -I$(python3 -c "import torch, os; print(os.path.join(torch.__path__[0], 'include'))") \
  -I$(python3 -c "import torch, os; print(os.path.join(torch.__path__[0], 'include/torch/csrc/api/include'))") \
  -L$(python3 -c "import site; print(site.getsitepackages()[0])")/torch/lib \
  -L/usr/local/nvidia/lib64 \
  -ltorch -ltorch_cpu -ltorch_python -lc10 -lcuda \
  -Xlinker -rpath=$(python3 -c "import site; print(site.getsitepackages()[0])")/torch/lib

# Check if it worked
!ls -lh optix_knn.so

-rwxr-xr-x 1 root root 2.8M Nov 29 10:37 optix_knn.so


In [28]:
# Restore the original bindings.cpp name
!mv bindings.cu bindings.cpp

# Verify the module works
!python3 -c "import sys; sys.path.insert(0, '.'); import optix_knn; print('✅ OptiX KNN module loaded successfully!')"

✅ OptiX KNN module loaded successfully!


In [29]:
# Navigate back to the genie root directory
%cd /kaggle/working/genie

# Install GENIE in editable mode
!pip install -e .

# Install Nerfstudio CLI
!ns-install-cli

/kaggle/working/genie
Obtaining file:///kaggle/working/genie
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for nerfstudio-genie (pyproject.toml) ... done
  Created wheel for nerfstudio-genie: filename=nerfstudio_genie-0.1.0-0.editable-py3-none-any.whl size=4110 sha256=1d9a90d13f9d0d087e15993c95e05c33554c8d55510cf0683a09524d434abaa5
  Stored in directory: /tmp/pip-ephem-wheel-cache-5or5pqc3/wheels/55/c7/11/fe4a9b621982ad9dfad1f4f074e0e5fc2e260ec3e9d6522a83
Successfully built nerfstudio-genie
[10:37:57] 🤷 .zshrc not found, skipping.                                                                 ]8;id=677364;file:///usr/local/lib/python3.11/dist-packages/nerfstudio/scripts/completions/install.py\install.py]8;;\:]8;id=765160;file:///usr/local/lib/python3.11/dist-packages/nerfstudio/scripts/completions/ins

In [30]:
# Test if GENIE commands are available
!ns-train --help | head -20

# Check if genie-export and genie-render are available
!which genie-export
!which genie-render

# Verify the OptiX KNN module is accessible
!python3 -c "import sys; sys.path.insert(0, 'genie/knn'); import optix_knn; print('✅ OptiX KNN module works!')"

2025-11-29 10:44:01.028105: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764413041.049617    1011 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764413041.055608    1011 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/usr/local/lib/python3.11/dist-packages/nerfstudio/field_components/activations.py:32: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd(cast_inputs=torch.float32)
/usr/local/lib/python3.11/dist-packages/nerfstudio/field_components/activations.py:38: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torc

In [31]:
# Find and fix the config file
!find /kaggle/working/genie -name "*config*.py" -type f | grep -i genie

# Let's check the genie_config.py file
!cat /kaggle/working/genie/genie/genie_config.py | grep -A 5 "milestones"

/kaggle/working/genie/genie/genie_config.py
